In [5]:
from docuverse.engines import SearchResult
from docuverse.engines import SearchEngine
from docuverse.engines.search_engine_config_params import DocUVerseConfig
from docuverse.utils import get_param
import pickle

In [6]:
config="experiments/zdocs/milvus_dense.granite-125m.test.yaml"
engine=SearchEngine(DocUVerseConfig(config))

Retrieval engine: milvus-dense
Running on the gpus:  ['NVIDIA GeForce RTX 5090']
=== done initializing model


In [7]:
results=engine.read_output()

In [8]:
res=results[0]

In [9]:
len(res)

46

In [10]:
urls=[[get_param(d, 'metadata.url') for d in res] for res in results]
qurls = [d.question.get('metadata')['gold-urls'] for d in results]
qtext = [d.question.text for d in results]

In [11]:
data=engine.read_data()

Cache filename is /home/raduf/.local/share/elastic_ingestion/benchmark__zdocs__corpus_fixed.jsonl.bz2_512_100_True_all_granite-embedding-125m-english.pickle.xz
Skipped 0 passages.


Computing statistics: 100%|██████████| 903792/903792 [00:01<00:00, 803989.86it/s]


Statistics:
Number of documents:903792    
Number of tiles:    903792    
#tiles per document:1.00      
                    Character-based:    Token-based:        
  Minimum length:   15                  6                   
  Maximum length:   2492                512                 
  Average length:   1070.9              244.9               
Char histogram:

# NumSamples = 903792; Min = 15.00; Max = 2492.00
# Mean = 1070.853898; Variance = 487834.385996; SD = 698.451420; Median 859.000000
# each ∎ represents a count of 2843
   15.0000 -   262.7000 [ 75287]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
  262.7000 -   510.4000 [213235]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
  510.4000 -   758.1000 [127280]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
  758.1000 -  1005.8000 [ 80451]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
 1005.8000 -  1253.5000 [ 59336]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
 1253.5000 -  1501.2000 [ 45657]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
 1501.2000 -  1748.9000 [ 35869]: ∎∎∎∎∎∎∎∎∎∎∎∎

In [12]:
url2id = {}
for d in data:
    url2id[d['metadata']['url']] = d['id']

In [14]:
from rouge_score.rouge_scorer import RougeScorer
rouge_scorer = RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
def compute_rouge_matches(result, thr=0.9):
    gold = result.question.get('metadata.answer')

    scores = []
    for answer in result:
        score = rouge_scorer.score(gold, answer.text)
        scores.append(score['rouge1'].recall)

    inds = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    # print(f"Best answer for {res.question.text}: {res[inds[0]]['id']}, recall: {scores[inds[0]]}")
    # return {result.question.id: {"best": result[inds[0]]['id'], "score": scores[inds[0]]}}
    res = []
    for ansid in inds:
        if scores[ansid] >= thr:
            res.append({'id': result[ansid]["id"], 'score': scores[ansid]})
    return res

In [15]:
compute_rouge_matches(res, 0.9)

[{'id': '451813522832071401-0-1261', 'score': 1.0}]

In [16]:
compute_rouge_matches(results[3])

[{'id': '451813522833601023-0-823', 'score': 1.0}]

In [24]:
import json

questions = [json.loads(s) for s in open("benchmark/zdocs/questions_all_fixed.jsonl").readlines()]

In [25]:
from tqdm.notebook import tqdm
from docuverse.utils import get_orig_docid
outq = questions.copy()
for i, q in tqdm(enumerate(outq), total=len(outq)):
    matches = compute_rouge_matches(results[i])
    if len(matches) > 0:
        q['metadata']['relevant'] = [get_orig_docid(m['id']) for m in matches]
        q['metadata']['rouge_scores'] = [m['score'] for m in matches]
    else:
        q['metadata']['rouge_scores'] = []
        q['metadata']['relevant'] = [url2id[q['metadata']['gold-urls']]] if q['metadata']['gold-urls'] in url2id else []

  0%|          | 0/510 [00:00<?, ?it/s]

In [21]:
outq[199]

{'metadata': {'gold-urls': 'https://www.ibm.com/docs/en/ztpf/2023?topic=zbxxx-zbevf-set-set-business-event-processing-control-values',
  'answer': 'Use the ZBEVF SET command with the MAXECBs parameter.',
  'fname': '/Users/vineet/data/systemz/data/predicted_ans_with_analysis_20250121/tpf_gds_2023-09-23.res.csv',
  'relevant': ['451813522831592632', '451813522833418860'],
  'rouge_scores': [1.0, 1.0]},
 '_id': 'tpf_gds_77',
 'text': 'How do I set the number of concurrent dispatch ECBs for a dispatch queue?'}

In [26]:
open("benchmark/zdocs/missing_urls.new.txt", "w").writelines([q['metadata']['gold-urls']+"\n" for q in outq if q['metadata']['relevant']==[] and q['metadata']['answer']!=""])

In [23]:
# Fix the questions' urls: remove tags
import json
qfile = "benchmark/zdocs/questions_all_fixed.jsonl"
qfile_fixed = "benchmark/zdocs/questions_all_fixed_new.jsonl"
qs = open(qfile).readlines()
with open(qfile_fixed, "w") as out:
    for line in qs:
        q = json.loads(line)
        url = q['metadata']['gold-urls']
        pos = url.find('#')
        if pos >= 0:
            q['metadata']['gold_urls'] = url[:pos]
        out.write(json.dumps(q)+"\n")